In [17]:
import json

In [6]:
import pandas as pd

def remove_unnecessary_timestamps(df):
    """
    Filters the dataframe to keep only the first and last records where latitude and longitude remain the same
    across consecutive timestamps for each maid.

    Parameters:
    df (pd.DataFrame): Input dataframe containing 'maid', 'datetime', 'latitude', and 'longitude' columns.

    Returns:
    pd.DataFrame: Filtered dataframe with only the first and last records where coordinates remain the same for each maid.
    """
    # Round latitude and longitude to 4 decimal places
    df['latitude'] = df['latitude'].astype(float).round(4)
    df['longitude'] = df['longitude'].astype(float).round(4)
    # Sort by 'maid' and 'datetime' to ensure consecutive timestamps are in order within each maid
    df = df.sort_values(by=['maid', 'datetime','latitude','longitude']).reset_index(drop=True)

    # Identify and keep first and last records where lat and long remain the same within each maid
    df['prev_lat'] = df.groupby('maid')['latitude'].shift(1)
    df['prev_long'] = df.groupby('maid')['longitude'].shift(1)
    df['next_lat'] = df.groupby('maid')['latitude'].shift(-1)
    df['next_long'] = df.groupby('maid')['longitude'].shift(-1)

    # Filter to keep rows where latitude or longitude changes within each maid
    filtered_df = df[(df['latitude'] != df['prev_lat']) |
                     (df['longitude'] != df['prev_long']) |
                     (df['latitude'] != df['next_lat']) |
                     (df['longitude'] != df['next_long'])]
    filtered_df.drop(['prev_lat', 'prev_long', 'next_lat', 'next_long'], axis=1, inplace=True)
    # Drop records for maids with only one record
    # Count occurrences of each maid
    maid_counts = df['maid'].value_counts()
    maid_single_records = maid_counts[maid_counts == 1].index
    filtered_df = filtered_df[~filtered_df['maid'].isin(maid_single_records)]

    return filtered_df



In [7]:
df= pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\all_visitors.csv')


In [12]:
# df['datetime']= pd.to_datetime(df['datetime'])
df['datetime'].dtype

dtype('<M8[ns]')

In [19]:
visitors = df['maid'].unique().tolist()
len(visitors)

140180

In [13]:
df_useful = remove_unnecessary_timestamps(df)
print(f'length of dataframe now: {len(df_useful)}')
# print(f'count of unique MAIDs: {df['maid'].nunique()}')
# len(df['maid'].unique())
df_useful['maid'].nunique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_22796\1288079991.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['prev_lat', 'prev_long', 'next_lat', 'next_long'], axis=1, inplace=True)


length of dataframe now: 4554069


134316

In [14]:
df_useful.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\useful_records.csv')

In [16]:
unique_useful_ids = df_useful['maid'].unique().tolist()
len(unique_useful_ids)

134316

In [18]:
with open(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\useful_ids.json', 'w') as f:
    json.dump(unique_useful_ids, f)

In [22]:
# non_useful_ids = list(set(visitors)-set(unique_useful_ids))
print(len(non_useful_ids))
with open(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\non_useful_ids.json','w') as f:
    json.dump(non_useful_ids,f)

5864


In [23]:
df_non_useful = df[df['maid'].isin(set(non_useful_ids))]

In [25]:
# df_non_useful.shape
df_non_useful.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\non_useful_filtered_records.csv')

In [33]:
sample_useful_ids = df_useful['maid'].sample(n=5, random_state =3).tolist()
sample_useful_ids

['eedfada4-2954-4e04-acbb-3f2f7c39c1af',
 'b7f4ac9c-5db1-4895-9ded-5338492fd8c2',
 '42274937-aa34-4624-9486-351f35e58eff',
 'ae679049-1a4f-4813-a194-fb3f69c5d582',
 '2cc2e858-b990-4ab1-a481-c61eba395cca']

In [34]:
sample_df_all_timestamps = df[df['maid'].isin(set(sample_useful_ids))]
sample_df_useful = df_useful[df_useful['maid'].isin(set(sample_useful_ids))]

In [36]:
sample_df_all_timestamps.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\sample_records_check\sample_df_all_timestamps.csv', index=False)
sample_df_useful.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Useful records\sample_records_check\sample_df_useful.csv', index=False)